<a href="https://colab.research.google.com/github/rabaouiyassine/Project/blob/main/yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd


In [ ]:

def trading_days_to_target(symbol, target_percent, start_date, end_date):
    """
    Calculate the number of trading days it takes for the stock to reach the target percent change.

    Args:
    - symbol (str): The stock ticker symbol.
    - target_percent (float): The target percent change (e.g., 10 for 10%).
    - start_date (str): Start date in the format 'YYYY-MM-DD'.
    - end_date (str): End date in the format 'YYYY-MM-DD'.

    Returns:
    int: Number of trading days.
    """
    # Fetch stock data for the given timeframe
    df = yf.download(symbol, start=start_date, end=end_date)

    # Calculate daily percentage change
    df['pct_change'] = df['Adj Close'].pct_change()

    # Calculate rolling cumulative sum of percentage change
    df['cum_pct_change'] = df['pct_change'].cumsum()

    # Find the index where the stock reaches the target percent change
    target_idx = df[df['cum_pct_change'].abs() >= target_percent / 100].index.min()

    if pd.isna(target_idx):
        return None  # Stock never reached the target within the available data

    start_date_idx = df.index.min()
    days_count = (df.index.get_loc(target_idx) - df.index.get_loc(start_date_idx))

    return days_count

def main():
    stock_symbols = ['AAPL', 'GOOGL', 'MSFT']
    target_percent = 2  # Set your desired target percent.Here we aim for a 2% change.
    start_date ='2023-09-10'
    end_date = '2023-09-19'

    results = {}
    for symbol in stock_symbols:
        days = trading_days_to_target(symbol, target_percent, start_date, end_date)
        if days is not None:
            results[symbol] = days
        else:
            results[symbol] = "Never reached target within the given timeframe."

    for symbol, days in results.items():
        print(f"{symbol}: {days} trading days to reach {target_percent}% change between {start_date} and {end_date}.")

if __name__ == "__main__":
    main()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
AAPL: 2 trading days to reach 2% change between 2023-09-10 and 2023-09-19.
GOOGL: Never reached target within the given timeframe. trading days to reach 2% change between 2023-09-10 and 2023-09-19.
MSFT: 4 trading days to reach 2% change between 2023-09-10 and 2023-09-19.


**Script 2**

Takes a a CSV or excel list and output CSV

Options Feature added

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def fetch_options(symbol, start_date):
    stock = yf.Ticker(symbol)
    today = datetime.strptime(start_date, '%Y-%m-%d')
    sixty_days_later = today + timedelta(days=60)

    valid_options = []

    stock_price = stock.history(period="1d", start=start_date).iloc[0]['Close']

    for expiration_date in stock.options:
        if datetime.strptime(expiration_date, '%Y-%m-%d') > sixty_days_later:
            option_chain = stock.option_chain(expiration_date)

            for opt_df in [option_chain.calls, option_chain.puts]:

                opt_df = opt_df[opt_df['openInterest'] > 200].copy()


                if not opt_df.empty:

                    opt_df.loc[:, 'stock_symbol'] = symbol
                    opt_df.loc[:, 'stock_price'] = stock_price
                    opt_df.loc[:, 'Risk'] = round(opt_df['lastPrice'] / stock_price, 3)

                    valid_options.append(opt_df)

    if not valid_options:
        return pd.DataFrame() # return an empty dataframe if there are no valid options

    valid_options_df = pd.concat(valid_options, ignore_index=True)

    # Sort by date and take the top 5 with the longest pricing history
    valid_options_df = valid_options_df.sort_values(by='lastTradeDate', ascending=True).head(5)

    return valid_options_df


def read_symbols_from_file(uploaded_files):
    filename = next(iter(uploaded_files))
    extension = filename.split('.')[-1]

    if extension == "csv":
        symbols_df = pd.read_csv(uploaded_files[filename])
    elif extension in ["xls", "xlsx"]:
        symbols_df = pd.read_excel(uploaded_files[filename])
    else:
        raise ValueError(f"Unsupported file format: {extension}")

    return symbols_df['Stock_Symbols'].tolist()


def download_csv(button, dataframe, columns):
    result_filename = "results.csv"
    dataframe[columns].to_csv(result_filename, index=False)
    files.download(result_filename)

def main():
    print("Upload your file (.xls or .csv): ")
    uploaded_files = files.upload()

    stock_symbols = read_symbols_from_file(uploaded_files)

    start_date = input("Enter the start date (format: YYYY-MM-DD, Press Enter for default date 2023-01-01): ")
    if not start_date:
        start_date = '2023-01-01'
    all_valid_options = []

    for symbol in stock_symbols:
        valid_options = fetch_options(symbol, start_date)
        all_valid_options.append(valid_options)

    all_options_df = pd.concat(all_valid_options, ignore_index=True)

    columns_to_print = ['stock_symbol', 'contractSymbol', 'expiration', 'lastPrice', 'stock_price', 'Risk', 'openInterest', 'lastTradeDate']
    columns_present = [col for col in columns_to_print if col in all_options_df.columns]

    # Create a button for downloading the CSV
    download_button = widgets.Button(description="Save CSV")
    download_button.on_click(lambda button: download_csv(button, all_options_df, columns_present))
    display(download_button)

if __name__ == "__main__":
    main()


Upload your file (.xls or .csv): 


Saving stock_symbols.xlsx to stock_symbols (7).xlsx
Enter the start date (format: YYYY-MM-DD, Press Enter for default date 2023-01-01): 


Button(description='Save CSV', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>